In [1]:
import pandas as pd
import numpy as np
import glob

In [3]:
df = pd.read_csv('estat_gov_10a_exp_from_2016.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   freq     99 non-null     object 
 1   unit     99 non-null     object 
 2   sector   99 non-null     object 
 3   cofog99  99 non-null     object 
 4   na_item  99 non-null     object 
 5   geo      99 non-null     object 
 6   2016     99 non-null     float64
 7   2017     99 non-null     float64
 8   2018     99 non-null     float64
 9   2019     99 non-null     float64
 10  2020     99 non-null     object 
 11  2021     99 non-null     object 
 12  2022     99 non-null     object 
dtypes: float64(4), object(9)
memory usage: 10.2+ KB


In [4]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t') for file in files}

In [5]:
for col in ["freq", "unit", "sector", "cofog99", "na_item", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,unit,sector,cofog99,na_item,country,2016,2017,2018,2019,2020,2021,2022
0,Annual,Million euro,General government,Health,Total general government expenditure,Austria,29223.8,30239.1,31770.2,32971.5,35132.8,40840.4,41735.6
1,Annual,Million euro,General government,Health,Total general government expenditure,Belgium,32788.8,33787.7,35116.6,36353.2,40151.0,43435.6,44951.9 p
2,Annual,Million euro,General government,Health,Total general government expenditure,Bulgaria,2414.3,2547.3,2786.0,2845.5,3600.0,4151.2,4767.3
3,Annual,Million euro,General government,Health,Total general government expenditure,Switzerland,13414.8,13002.7,13042.8,14085.7,16800.6,19011.3,18018.9
4,Annual,Million euro,General government,Health,Total general government expenditure,Cyprus,474.8,506.3,568.9,811.8,1306.0,1641.7,1720.2


In [6]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [7]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [8]:
df.head()

,freq,unit,sector,cofog99,na_item,country,2016,2017,2018,2019,2020,2021,2022
0,Annual,Million euro,General government,Health,Total general government expenditure,Austria,29223.0,30239.0,31770.0,32971.0,35132.0,40840.0,41735.0
1,Annual,Million euro,General government,Health,Total general government expenditure,Belgium,32788.0,33787.0,35116.0,36353.0,40151.0,43435.0,44951.0
2,Annual,Million euro,General government,Health,Total general government expenditure,Bulgaria,2414.0,2547.0,2786.0,2845.0,3600.0,4151.0,4767.0
3,Annual,Million euro,General government,Health,Total general government expenditure,Switzerland,13414.0,13002.0,13042.0,14085.0,16800.0,19011.0,18018.0
4,Annual,Million euro,General government,Health,Total general government expenditure,Cyprus,474.0,506.0,568.0,811.0,1306.0,1641.0,1720.0


In [9]:
for col in df.columns:
    if not col.isdigit() and df[col].nunique() == 1:
        df.drop(columns=[col], inplace=True)

In [10]:
df.head()

,unit,cofog99,country,2016,2017,2018,2019,2020,2021,2022
0,Million euro,Health,Austria,29223.0,30239.0,31770.0,32971.0,35132.0,40840.0,41735.0
1,Million euro,Health,Belgium,32788.0,33787.0,35116.0,36353.0,40151.0,43435.0,44951.0
2,Million euro,Health,Bulgaria,2414.0,2547.0,2786.0,2845.0,3600.0,4151.0,4767.0
3,Million euro,Health,Switzerland,13414.0,13002.0,13042.0,14085.0,16800.0,19011.0,18018.0
4,Million euro,Health,Cyprus,474.0,506.0,568.0,811.0,1306.0,1641.0,1720.0


In [11]:
df = df.rename(columns={'cofog99': 'sector'})
df.head()

,unit,sector,country,2016,2017,2018,2019,2020,2021,2022
0,Million euro,Health,Austria,29223.0,30239.0,31770.0,32971.0,35132.0,40840.0,41735.0
1,Million euro,Health,Belgium,32788.0,33787.0,35116.0,36353.0,40151.0,43435.0,44951.0
2,Million euro,Health,Bulgaria,2414.0,2547.0,2786.0,2845.0,3600.0,4151.0,4767.0
3,Million euro,Health,Switzerland,13414.0,13002.0,13042.0,14085.0,16800.0,19011.0,18018.0
4,Million euro,Health,Cyprus,474.0,506.0,568.0,811.0,1306.0,1641.0,1720.0


In [14]:
# Create a new dataframe with the required structure
refactored_df = pd.DataFrame(columns=['country', 'year', 'total', 'health', 'percentage'])

# Iterate over each row in the original dataframe

totals_df = df[df['sector']=="Total"]
health_df = df[df['sector']=="Health"]
percentage_df = df[df['unit']=="Percentage of total"]

rows = []

for country in df['country'].unique():
	country_totals = totals_df[totals_df['country'] == country]
	country_health = health_df[health_df['country'] == country]
	country_percentage = percentage_df[percentage_df['country'] == country]
	for year in year_cols:
		total = country_totals[year].values[0] if not country_totals.empty else np.nan
		health = country_health[year].values[0] if not country_health.empty else np.nan
		percentage = country_percentage[year].values[0] if not country_percentage.empty else np.nan
		rows.append({'country': country, 'year': year, 'total': total, 'health': health, 'percentage': percentage})

refactored_df = pd.concat([refactored_df, pd.DataFrame(rows)], ignore_index=True)

# Drop duplicate rows
print(f"Number of duplicate rows: {refactored_df.duplicated(subset=['country', 'year']).sum()}")
refactored_df.drop_duplicates(subset=['country', 'year'], inplace=True)

# Reset index
refactored_df.reset_index(drop=True, inplace=True)

refactored_df.head()

Number of duplicate rows: 0


,country,year,total,health,percentage
0,Austria,2016,179059.0,29223.0,16.0
1,Austria,2017,182091.0,30239.0,16.0
2,Austria,2018,187850.0,31770.0,16.0
3,Austria,2019,193308.0,32971.0,17.0
4,Austria,2020,216367.0,35132.0,16.0


In [26]:
health = refactored_df[(refactored_df['country'] == 'Belgium') & (refactored_df['year']=='2016')].health.values[0]
total = refactored_df[(refactored_df['country'] == 'Belgium') & (refactored_df['year']=='2016')].total.values[0]
perc = refactored_df[(refactored_df['country'] == 'Belgium') & (refactored_df['year']=='2016')].percentage.values[0]
print(health, total, health*100/total, perc)

32788.0 228451.0 14.352311874318781 14.0


In [18]:
refactored_df.to_csv("./clean/estat_gov_10a_exp_from_2016.csv", index=False)